<a href="https://colab.research.google.com/github/roshanis/notebooks/blob/main/babyllama_finetuning4bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama2-7b Fine-Tuning with 4bit Quantization

We recommend using a GPU runtime for this example. In the Colab menu bar, choose Runtime > Change Runtime Type and choose GPU under Hardware Accelerator.

## Install Ludwig

We'll use the latest version of Ludwig which includes support for quantized fine-tuning.

In [2]:
!pip install ludwig
!pip install fastapi kaleido python-multipart uvicorn cohere tokenizers transformers tiktoken tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 55.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.


## Set your HuggingFace API Token

Obtain a [HuggingFace API Token](https://huggingface.co/docs/hub/security-tokens) and request access to [Llama2-7b-hf](https://huggingface.co/meta-llama/Llama-2-7b-hf) before proceeding.

In [ ]:
import os

os.environ["HUGGING_FACE_HUB_TOKEN"] = "<api_token>"

## Configure Model Training

The Ludwig [configuration](https://ludwig.ai/latest/configuration/) specifies the components of the training job including:

- Model type (LLM) and base pretrained model name from HuggingFace
- Input and output features from the training dataset
- Quantization (4bit) and parameter-efficient fine-tuning (LoRA)
- Training hyperparameters (learning rate, batch size, etc.)
- Preprocessing (e.g., sampling to speed up training)
- Backend for execution (local, but could also be Ray)

In [ ]:
import yaml

config_str = """
model_type: llm
base_model: meta-llama/Llama-2-7b-hf

quantization:
  bits: 4

adapter:
  type: lora

prompt:
  template: |
    ### Instruction:
    {instruction}

    ### Input:
    {input}

    ### Response:

input_features:
  - name: prompt
    type: text
    preprocessing:
      max_sequence_length: 256

output_features:
  - name: output
    type: text
    preprocessing:
      max_sequence_length: 256

trainer:
  type: finetune
  learning_rate: 0.0001
  batch_size: 1
  gradient_accumulation_steps: 16
  epochs: 3
  learning_rate_scheduler:
    warmup_fraction: 0.01

preprocessing:
  sample_ratio: 0.1
"""

config = yaml.safe_load(config_str)

## Train!

Start training on your local GPU and monitor progress (including metrics) inline.

In this example, we'll be training on the [Alpaca](https://crfm.stanford.edu/2023/03/13/alpaca.html) dataset to turn Llama2-7b into a rudimentary chatbot. But you can use any dataset to fine-tune for other tasks.

In [ ]:
import logging
from ludwig.api import LudwigModel


model = LudwigModel(config=config, logging_level=logging.INFO)
results = model.train(dataset="ludwig://alpaca")
print(results)

## What's next?

Now you can save / export your model to HuggingFace or explore [Predibase](https://predibase.com/) for serving and managed infrastructure for training larger LLMs and other model types.